In [ ]:
%pip install pandas, matplotlib, tqdm, neo4j

In [ ]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "your_password"))

In [ ]:
import pandas as pd

actions_df = pd.read_csv("mooc_actions.tsv", sep="\t", header=None, names=["user_id", "target_id", "action_id", "timestamp"])
features_df = pd.read_csv("mooc_action_features.tsv", sep="\t", header=None, names=["feature1", "feature2", "feature3", "feature4"])
labels_df = pd.read_csv("mooc_action_labels.tsv", sep="\t", header=None, names=["label"])

# Combine dataframes
combined_df = pd.concat([actions_df, features_df, labels_df], axis=1)

In [ ]:
def insert_data(tx, user_id, target_id, action_id, timestamp, feature1, feature2, feature3, feature4, label):
    tx.run("""
        MERGE (u:User {id: $user_id})
        MERGE (t:Target {id: $target_id})
        CREATE (u)-[:PERFORMS {
            action_id: $action_id,
            timestamp: $timestamp,
            feature1: $feature1,
            feature2: $feature2,
            feature3: $feature3,
            feature4: $feature4,
            label: $label
        }]->(t)
    """, user_id=user_id, target_id=target_id, action_id=action_id, timestamp=timestamp,
         feature1=feature1, feature2=feature2, feature3=feature3, feature4=feature4, label=label)

with driver.session() as session:
    for index, row in combined_df.iterrows():
        session.write_transaction(insert_data, row['user_id'], row['target_id'], row['action_id'],
                                  row['timestamp'], row['feature1'], row['feature2'],
                                  row['feature3'], row['feature4'], row['label'])

In [ ]:
def run_query(tx, query):
    result = tx.run(query)
    for record in result:
        print(record)

### CYPHER QUERIES EXECUTION

In [ ]:
with driver.session() as session:
    session.read_transaction(run_query)